# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [2]:
VOCAB_SIZE = 2500
TRAINING_DATA_FILE = "./SpamData/02_Training/train-data.txt"
TEST_DATA_FILE = "./SpamData/02_Training/test-data.txt"

TOKEN_SPAM_PROB_FILE = "./SpamData/03_Testing/prob-spam.txt"
TOKEN_HAM_PROB_FILE = "./SpamData/03_Testing/prob-nonspam.txt"
TOKEN_ALL_PROB_FILE = "./SpamData/03_Testing/prob-all-tokens.txt"

TEST_FEATURE_MATRIX = "./SpamData/03_Testing/test-features.txt"
TEST_TARGET_FILE = "./SpamData/03_Testing/test-target.txt"


# Read and Load Features from the .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_test_data[-5:]

array([[5793,  831,    0,    1],
       [5793, 1093,    0,    1],
       [5793, 1126,    0,    1],
       [5793, 1254,    0,    1],
       [5793, 1998,    0,    1]])

In [6]:
sparse_train_data[-5:]

array([[5795, 1841,    0,    1],
       [5795, 1888,    0,    2],
       [5795, 2112,    0,    1],
       [5795, 2259,    0,    1],
       [5795, 2461,    0,    1]])

In [7]:
print('Number of rows in training file', sparse_train_data.shape[0])
print('Number of rows in test file', sparse_test_data.shape[0])

Number of rows in training file 265558
Number of rows in test file 110567


In [8]:
print('Number of emails in training file', np.unique(sparse_train_data[:, 0]).size)

Number of emails in training file 4014


In [9]:
print('Number of emails in testing file', np.unique(sparse_test_data[:, 0]).size)

Number of emails in testing file 1723


## How to create an Empty DataFrame

In [10]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [11]:
len(column_names)

2502

In [12]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [13]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [14]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full Matrix from a Sparse Matrix

In [64]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)

    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]

        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence

    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [16]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 11.4 s, sys: 126 ms, total: 11.6 s
Wall time: 11.6 s


In [17]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes model

## Calculating the Probability of Spam

In [18]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print("Probability of spam is", prob_spam)

Probability of spam is 0.3111609367214748


## Total Number of Words / Tokens

In [19]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4014,)

In [21]:
email_lengths[:5]

DOC_ID
0     50
1     76
2     87
3     76
4    137
dtype: int64

In [22]:
total_wc = email_lengths.sum()
total_wc

444959

## Number of Tokens in Spam and Ham Emails

In [23]:
spam_lengths = full_train_features[full_train_data['CATEGORY'] == 1].sum(axis=1)
spam_lengths.shape

(1249,)

In [24]:
total_spam = spam_lengths.sum()
total_spam

193993

In [25]:
ham_lengths = full_train_features[full_train_data['CATEGORY'] == 0].sum(axis=1)
ham_lengths.shape

(2765,)

In [26]:
total_ham = ham_lengths.sum()
total_ham

250966

In [27]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [28]:
nonspam_wc = ham_lengths.sum()
spam_wc = spam_lengths.sum()

print(f"{nonspam_wc=}, {spam_wc=}")

nonspam_wc=250966, spam_wc=193993


In [29]:
spam_wc + nonspam_wc - total_wc

0

In [30]:
print('Average number of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average number of words in ham emails {:.0f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average number of words in spam emails 155
Average number of words in ham emails 91


## Summing the Tokens Occuring in Spam

In [31]:
full_train_features.shape

(4014, 2500)

In [32]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY ==1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [33]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,2,0,0,3,2,2,2,2,1,0,...,0,0,0,0,0,0,0,0,0,0
1887,0,0,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,2,0,1,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,3,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1895,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [34]:
train_spam_tokens.shape

(1249, 2500)

In [35]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [36]:
summed_spam_tokens.tail()

2495    21
2496    17
2497     5
2498     7
2499    14
dtype: int64

## Summing the Tokens Occuring in Ham

In [37]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [38]:
summed_ham_tokens.shape

(2500,)

In [39]:
summed_ham_tokens.tail()

2495     7
2496    15
2497    27
2498    21
2499    16
dtype: int64

In [40]:
train_ham_tokens[2499].sum() + 1

16

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [41]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.009064
1    0.004926
2    0.006881
3    0.010662
4    0.006916
dtype: float64

In [42]:
prob_tokens_spam.sum()

0.9999999999999998

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [43]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.0

## P(Token) - Probability that Token Occurs

In [44]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [45]:
prob_tokens_all.sum()

1.0

# Save the Trained Model

In [46]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [47]:
sparse_test_data.shape

(110567, 4)

In [65]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

CPU times: user 5.26 s, sys: 50.3 ms, total: 5.31 s
Wall time: 5.4 s


In [66]:
sparse_test_data.shape

(110567, 4)

In [67]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [68]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)

In [69]:
full_test_data.shape

(1723, 2501)

In [70]:
full_test_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
12,1,0,2,1,1,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
15,1,3,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
17,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
19,1,1,2,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
